In [1]:
# Imports
import numpy as np #Mathematical tools
import matplotlib.pyplot as plt #Plots charts
import pandas as pd #Import and manage data sets
from sklearn.neural_network import MLPClassifier 
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
# Preprocessing library
from sklearn.preprocessing import Imputer
# Categorical data encoder library
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Splitting the data set into training and testing sets library
from sklearn.model_selection import train_test_split
#Feature scaling library
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [2]:
# Imports part 2

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
# Functions part 1

def best_output(row):
    # se o downloadTime é diferente de 100 para T2 e T3, ambos os targets completam o download
    if ((row.downloadTimeT2!=100)&(row.downloadTimeT3!=100)):
        # o melhor output será aquele para o qual o downloadTime é menor
        if (row.downloadTimeT2<=row.downloadTimeT3):
            return 0
        else:
            return 1
    
    # se o downloadTime é diferente de 100 apenas para um dos targets, só um deles completa o download
    elif ((row.downloadTimeT2!=100)|(row.downloadTimeT3!=100)):
        # o melhor output será aquele para o qual o downloadTime é diferente de 100
        if (row.downloadTimeT2!=100):
            return 0
        else:
            return 1
        
    # se o downloadTime é igual a 100 para T2 e T3, ambos os targets não completam o download
    elif ((row.downloadTimeT2==100)&(row.downloadTimeT3==100)):
        # o melhor output será aquele para o qual o rxBytes é maior
        if (row.rxBytesT2>=row.rxBytesT3):
            return 0
        else:
            return 1
        
def download_complete(row, column):
    if (row[column]==0):
        if(row.downloadTimeT2!=100.0):
            return 1
        else:
            return 0
    else:
        if(row.downloadTimeT3!=100.0):
            return 1
        else: 
            return 0

        
def download_time(row, column):
    if ((row[column]==0)&(row.downloadTimeT2<100)):
        return row.downloadTimeT2
    elif ((row[column]==1)&(row.downloadTimeT3<100)):
        return row.downloadTimeT3

    
def throughput(row, column):
    if (row[column]==0):
        tp = (row.rxBytesT2/row.downloadTimeT2)*8/1e6
        return tp
    else:
        tp = (row.rxBytesT3/row.downloadTimeT3)*8/1e6
        return tp
    
def optimum_choice(row, column):
    if (row[column]==row.best_output):
        return 1
    else:
        return 0
    
def a2a4rsrp(row):
    if (row.rsrp2>=row.rsrp3):
        return 0
    else:
        return 1

In [ ]:
# Functions part 2
def percentageOfErrors(y_real, y_predicted):
    p= ((y_predicted != y_real).sum()/ y_real.size)
    return p 

def applyingClassifier (classifier,x_test, x_train, y_test, y_train):
    classifier.fit(x_train, y_train)
    # Predicting the test set results
    y_pred = classifier.predict(x_test)

    # Evaluation of the model
    # Making the confusion matrix
    #cm = confusion_matrix(y_test, y_pred)
    p=percentageOfErrors(y_real=y_test, y_predicted=y_pred)
    print(p)
    return p

def gettingClassifierConfiguration(classifier, param_test,x_train, y_train ):
    conf=GridSearchCV(estimator=classifier, param_grid=param_test, scoring='accuracy', cv=10, n_jobs=-1)
    conf.fit(x_train, y_train)
    print(conf.best_estimator_)
    return conf.best_estimator_

def gettingAllClassifiersConfigurations(x_train, y_train, randomState):
    # All test parameters
    param_test_knn = [{'n_neighbors':[(i) for i in range(5,31)], 'p':[1,2],'weights':['uniform', 'distance'],
                  'algorithm':['auto', 'brute']},{'n_neighbors':[(i) for i in range(1,21)], 'p':[1,2],
                 'weights':['uniform', 'distance'],
                  'algorithm':['ball_tree', 'kd_tree'], 'leaf_size':[10,20,30,40,50,60,70, 80,90,100]} ]
    param_test_svm =[{'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'degree':[(i) for i in range(1,21)],'kernel':['poly'],'decision_function_shape':['ovo','ovr'],
                  'shrinking':[True, False]},
                 {'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'kernel':['linear'],'decision_function_shape':['ovo','ovr'],
                  'shrinking':[True, False]},
              {'C':[1,2,3,4,5,6,7,8,9,10,100,200, 300, 400, 500, 600, 700, 800, 900, 1000],'kernel':['rbf', 'sigmoid'],'gamma':[1,0.5,0.1, 0.01,0.001, 0.0001, 0.0000001],
                 'decision_function_shape':['ovo','ovr'], 'shrinking':[True, False]}] 
    # Creating the DT classifier parameters
    param_test_dt=[{'criterion': ['gini', 'entropy'], 'splitter':['best', 'random'], 
                    'max_features':[None,'auto', 'sqrt', 'log2'], 'class_weight':['balanced', None]}]    
   
    # Creating the RF classifier parameters
    param_test_rf = {'n_estimators':[(i) for i in range(5,31)], 'criterion': ['gini', 'entropy'], 
                     'max_features':[None,'auto', 'sqrt', 'log2'], 'class_weight':['balanced', None], 
                     'verbose':[0], 'n_jobs':[-1], 'bootstrap':[True, False]}
    
    param_test_nn = [{'activation' : ['identity', 'logistic', 'tanh', 'relu'], 'solver' :['lbfgs'],
                'learning_rate' : ['constant', 'invscaling', 'adaptive'], 'hidden_layer_sizes': [[(10*i), (10*i), (10*i)] for i in range(1,6)],
                     'max_iter':[2000]},{'activation' : ['identity', 'logistic', 'tanh', 'relu'], 'solver' :['sgd', 'adam'],
                'learning_rate' : ['constant', 'invscaling', 'adaptive'], 'hidden_layer_sizes': [[(10*i), (10*i), (10*i)] for i in range(1,6)],
                     'max_iter':[2000], 'early_stopping':[True, False], 'validation_fraction':[0.1,0.15,0.2],'shuffle':[True, False],
                                         'learning_rate_init':[0.001,0.01, 0.1]}]
    # All classifiers
    classifier_knn=KNeighborsClassifier(metric = 'minkowski')
    classifier_svm=SVC(random_state=randomState)
    classifier_dt=DecisionTreeClassifier(random_state=randomState)
    classifier_rf=RandomForestClassifier(random_state=randomState)
    classifier_nn=MLPClassifier(random_state=randomState)
    
    #Finding the best configurations
    
    print("Best parameters for NN Classifier")
    conf_nn=gettingClassifierConfiguration(classifier=classifier_nn, param_test=param_test_nn,
                                           x_train=x_train, y_train=y_train)
    
    print("Best parameters for KNN Classifier")
    conf_knn=gettingClassifierConfiguration(classifier=classifier_knn, param_test=param_test_knn, 
                                            x_train=x_train, y_train=y_train)
    
    print("Best parameters for SVM Classifier")
    conf_svm=gettingClassifierConfiguration(classifier=classifier_svm, param_test=param_test_svm,
                                            x_train=x_train, y_train=y_train)
    print("Best parameters for DT Classifier")
    conf_dt=gettingClassifierConfiguration(classifier=classifier_dt, param_test=param_test_dt,
                                           x_train=x_train, y_train=y_train)
    print("Best parameters for RF Classifier")
    conf_rf=gettingClassifierConfiguration(classifier=classifier_rf, param_test=param_test_rf,
                                           x_train=x_train, y_train=y_train)
    
    
    conf=[conf_knn,conf_svm, conf_dt, conf_rf, conf_nn]
    return conf

def theBestScore(oldBest, candidate):
    if oldBest<candidate:
        return oldBest
    else:
        return candidate    
def theWorstScore(oldWorst, candidate):
    if oldWorst>candidate:
        return oldWorst
    else:
        return candidate
    
    
def applyingAllClassifiers (classifiers,numberOfRandomStates,x, y):   
    # Initializing the variables
    # Averages
    p_svm=0
    p_dt=0
    p_rf=0
    p_knn=0
    p_nn=0
    # Worst cases 
    w_knn=0
    w_svm=0
    w_rf=0
    w_dt=0
    w_nn=0
    # Best cases
    b_svm=1
    b_dt=1
    b_rf=1
    b_knn=1
    b_nn=1
    
    for (i) in range(0,numberOfRandomStates):
        r1=np.random.randint(1,10001)
        r2=np.random.randint(1,10001)
        # Splitting test data and train data
        x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = r1)
        # Feature scaling (normalizing or stadardization of the scales)
        # Helps the conversion of the algorithm
        sc_X = StandardScaler()
        x_train = sc_X.fit_transform(x_train)
        x_test = sc_X.transform(x_test) # There is no need to fit after the training set is fit        
        print("{}: RandomState r1={} and r2={}".format(i, r1,r2))
        # Fitting the KNN to the training set
        p=applyingClassifier(classifier=classifiers[0],x_test=x_test, x_train=x_train, y_test=y_test, y_train= y_train)
        p_knn=p_knn+p
        w_knn=theWorstScore(oldWorst=w_knn, candidate=p)
        b_knn=theBestScore(oldBest=b_knn, candidate=p)
        # Fitting the SVM to the training set
        classifiers[1].randomState=r2
        p=applyingClassifier(classifier=classifiers[1],x_test=x_test, x_train=x_train, y_test=y_test, y_train= y_train)
        p_svm=p_svm+p
        w_svm=theWorstScore(oldWorst=w_svm, candidate=p)
        b_svm=theBestScore(oldBest=b_svm, candidate=p)
        # Fitting the DT to the training set
        classifiers[2].randomState=r2
        p=applyingClassifier(classifier=classifiers[2],x_test=x_test,x_train=x_train,y_test=y_test,y_train=y_train)
        p_dt=p_dt+p
        w_dt=theWorstScore(oldWorst=w_dt, candidate=p)
        b_dt=theBestScore(oldBest=b_dt, candidate=p)
        # Fitting the RF to the training set
        classifiers[3].randomState=r2
        p=applyingClassifier(classifier=classifiers[3],x_test=x_test,x_train=x_train,y_test=y_test,y_train=y_train)
        p_rf=p_rf+p
        w_rf=theWorstScore(oldWorst=w_rf, candidate=p)
        b_rf=theBestScore(oldBest=b_rf, candidate=p)
        # Fitting the NN to the training set
        p=applyingClassifier(classifier=classifiers[4],x_test=x_test,x_train=x_train,y_test=y_test,y_train=y_train)
        p_nn=p_nn+p
        w_nn=theWorstScore(oldWorst=w_nn, candidate=p)
        b_nn=theBestScore(oldBest=b_nn, candidate=p)
    p_knn=p_knn/numberOfRandomStates
    p_svm=p_svm/numberOfRandomStates
    p_dt=p_dt/numberOfRandomStates
    p_rf=p_rf/numberOfRandomStates
    p_nn=p_nn/numberOfRandomStates
    
    p=np.array([[p_knn, p_svm, p_dt, p_rf, p_nn],[ b_knn, b_svm, b_dt, b_rf, b_nn], [w_knn, w_svm, w_dt, w_rf, w_nn]])
    return p   
    

# Sem shadowing

In [5]:
# Importing data

# importando os datasets
t2 = pd.read_csv('resultados/t2_OkumuraHata_Modificado', delimiter='\t')
t3 = pd.read_csv('resultados/t3_OkumuraHata_Modificado', delimiter='\t')

# garantindo que utilizaremos apenas as sementes presentes nos dois datasets
t2 = t2[t2.nRun.isin(t3.nRun)]
t3 = t3[t3.nRun.isin(t2.nRun)]
t2 = t2.reset_index(drop=True)
t3 = t3.reset_index(drop=True)

# combinando os datasets
data = t2
data = data.drop(['targetCellId', 'downloadTime', 'rxBytes'], axis=1)
data['downloadTimeT2'] = t2.downloadTime
data['downloadTimeT3'] = t3.downloadTime
data['rxBytesT2'] = t2.rxBytes
data['rxBytesT3'] = t3.rxBytes

data['best_output'] = data.apply(best_output, axis=1)
data.head()

# Splitting the data between independent variables and dependent variable
y = data['best_output']
x = data[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]

# Splitting test data and train data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
# Feature scaling (normalizing or stadardization of the scales)
# Helps the conversion of the algorithm
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test) # There is no need to fit after the training set is fit

In [12]:
conf=gettingAllClassifiersConfigurations(x_train=x_train, y_train=y_train, randomState=0)

# Setting the classifiers
classifier_knn=conf[0]
classifier_svm=conf[1]
classifier_dt=conf[2]
classifier_rf=conf[3]
classifier_nn=conf[4]
#classifier_knn=KNeighborsClassifier(algorithm='auto',n_neighbors=5, p=1, weights='uniform')
#classifier_svm=SVC(C=200, decision_function_shape='ovo', degree=3, kernel='poly', shrinking=True)
#classifier_dt=DecisionTreeClassifier(class_weight= 'balanced', criterion= 'gini', max_features= None, splitter= 'best')
#classifier_rf=RandomForestClassifier(bootstrap= True, class_weight='balanced', criterion= 'gini', 
#                                     max_features= None, n_estimators= 2, n_jobs= -1, verbose= 0)


classifiers=[classifier_knn,classifier_svm,classifier_dt,classifier_rf, classifier_nn]

Best parameters for NN Classifier
MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[10, 10, 10], learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
Best parameters for KNN Classifier
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=1,
           weights='uniform')
Best parameters for SVM Classifier
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)
Best parameters for DT Classifier
DecisionTreeClassifier(class_weight='b

In [13]:
# Applying the classifiers
maxRange=1000
p = applyingAllClassifiers(numberOfRandomStates=maxRange, x=x, y=y, classifiers=classifiers)

0: RandomState r1=1612 and r2=9219
0.0
0.0
0.005
0.005
0.0
1: RandomState r1=5290 and r2=3582
0.0
0.0
0.005
0.0
0.0
2: RandomState r1=9184 and r2=9021
0.005
0.005
0.01
0.01
0.005
3: RandomState r1=582 and r2=9346
0.0
0.0
0.0
0.0
0.0
4: RandomState r1=3393 and r2=4715
0.0
0.0
0.0
0.0
0.0
5: RandomState r1=5519 and r2=7869
0.005
0.005
0.0
0.005
0.005
6: RandomState r1=6261 and r2=8856
0.0
0.0
0.005
0.0
0.0
7: RandomState r1=7095 and r2=1690
0.0
0.0
0.005
0.0
0.0
8: RandomState r1=2911 and r2=6996
0.0
0.0
0.005
0.005
0.0
9: RandomState r1=2455 and r2=2920
0.005
0.005
0.005
0.005
0.005
10: RandomState r1=5020 and r2=191
0.0
0.0
0.005
0.005
0.0
11: RandomState r1=92 and r2=6727
0.0
0.0
0.0
0.0
0.0
12: RandomState r1=636 and r2=1540
0.0
0.0
0.005
0.005
0.0
13: RandomState r1=2842 and r2=6828
0.005
0.005
0.0
0.005
0.005
14: RandomState r1=647 and r2=6639
0.005
0.005
0.005
0.005
0.005
15: RandomState r1=4264 and r2=7907
0.0
0.0
0.0
0.0
0.0
16: RandomState r1=4477 and r2=1410
0.005
0.005
0.005


0.005
0.005
135: RandomState r1=4544 and r2=8187
0.01
0.0
0.0
0.0
0.0
136: RandomState r1=6032 and r2=4312
0.0
0.0
0.0
0.0
0.0
137: RandomState r1=7532 and r2=6201
0.01
0.005
0.005
0.005
0.005
138: RandomState r1=413 and r2=4403
0.0
0.0
0.0
0.0
0.0
139: RandomState r1=4151 and r2=6899
0.005
0.005
0.005
0.005
0.005
140: RandomState r1=6649 and r2=8686
0.0
0.0
0.0
0.0
0.0
141: RandomState r1=2728 and r2=3975
0.0
0.0
0.0
0.0
0.0
142: RandomState r1=6191 and r2=8796
0.0
0.0
0.005
0.005
0.0
143: RandomState r1=5448 and r2=2730
0.01
0.005
0.005
0.005
0.005
144: RandomState r1=5283 and r2=6732
0.0
0.0
0.0
0.0
0.0
145: RandomState r1=8873 and r2=918
0.0
0.0
0.005
0.005
0.005
146: RandomState r1=8108 and r2=5902
0.0
0.0
0.0
0.0
0.0
147: RandomState r1=3704 and r2=6822
0.005
0.0
0.005
0.0
0.0
148: RandomState r1=9173 and r2=5748
0.01
0.01
0.005
0.01
0.01
149: RandomState r1=9360 and r2=4200
0.005
0.0
0.0
0.0
0.0
150: RandomState r1=7179 and r2=9997
0.0
0.0
0.005
0.0
0.0
151: RandomState r1=3854 

0.005
0.0
270: RandomState r1=5690 and r2=1350
0.01
0.0
0.0
0.005
0.0
271: RandomState r1=1526 and r2=2500
0.01
0.01
0.005
0.01
0.01
272: RandomState r1=4315 and r2=6260
0.01
0.005
0.01
0.01
0.0
273: RandomState r1=9586 and r2=5514
0.0
0.0
0.0
0.0
0.0
274: RandomState r1=3072 and r2=5355
0.0
0.0
0.005
0.0
0.0
275: RandomState r1=257 and r2=9999
0.005
0.0
0.0
0.0
0.005
276: RandomState r1=1505 and r2=9351
0.01
0.005
0.005
0.005
0.005
277: RandomState r1=8364 and r2=4593
0.005
0.005
0.005
0.005
0.005
278: RandomState r1=5355 and r2=1800
0.005
0.005
0.005
0.005
0.005
279: RandomState r1=1384 and r2=179
0.005
0.0
0.0
0.0
0.0
280: RandomState r1=4935 and r2=804
0.0
0.0
0.005
0.005
0.0
281: RandomState r1=791 and r2=4737
0.01
0.01
0.01
0.01
0.01
282: RandomState r1=4006 and r2=8625
0.0
0.0
0.005
0.005
0.0
283: RandomState r1=2653 and r2=6114
0.0
0.0
0.0
0.0
0.0
284: RandomState r1=9768 and r2=2834
0.0
0.0
0.005
0.005
0.0
285: RandomState r1=2665 and r2=3496
0.0
0.0
0.0
0.0
0.0
286: RandomSta

0.01
0.0
406: RandomState r1=515 and r2=6892
0.005
0.005
0.005
0.005
0.005
407: RandomState r1=637 and r2=7706
0.0
0.0
0.005
0.005
0.0
408: RandomState r1=169 and r2=1902
0.005
0.005
0.005
0.005
0.005
409: RandomState r1=3518 and r2=4334
0.005
0.0
0.0
0.0
0.0
410: RandomState r1=5428 and r2=7792
0.0
0.0
0.01
0.01
0.0
411: RandomState r1=3019 and r2=3507
0.0
0.0
0.0
0.0
0.0
412: RandomState r1=130 and r2=30
0.0
0.0
0.0
0.0
0.0
413: RandomState r1=1582 and r2=6656
0.0
0.0
0.0
0.0
0.0
414: RandomState r1=8107 and r2=5072
0.01
0.01
0.01
0.01
0.01
415: RandomState r1=4756 and r2=9253
0.0
0.0
0.0
0.0
0.0
416: RandomState r1=6852 and r2=1904
0.005
0.005
0.005
0.005
0.005
417: RandomState r1=1292 and r2=6695
0.005
0.0
0.01
0.01
0.0
418: RandomState r1=4776 and r2=6017
0.005
0.005
0.005
0.005
0.005
419: RandomState r1=4033 and r2=59
0.0
0.0
0.0
0.0
0.0
420: RandomState r1=4907 and r2=9858
0.0
0.0
0.005
0.0
0.0
421: RandomState r1=6107 and r2=8152
0.0
0.0
0.005
0.005
0.01
422: RandomState r1=180

0.0
0.0
540: RandomState r1=625 and r2=9943
0.0
0.0
0.005
0.005
0.0
541: RandomState r1=8141 and r2=94
0.0
0.0
0.005
0.005
0.0
542: RandomState r1=4225 and r2=3259
0.005
0.005
0.005
0.005
0.005
543: RandomState r1=7428 and r2=5229
0.0
0.0
0.005
0.0
0.005
544: RandomState r1=9711 and r2=8471
0.005
0.0
0.0
0.0
0.0
545: RandomState r1=4971 and r2=1179
0.0
0.0
0.0
0.0
0.0
546: RandomState r1=8864 and r2=4848
0.005
0.005
0.005
0.005
0.005
547: RandomState r1=7796 and r2=8166
0.0
0.0
0.0
0.0
0.0
548: RandomState r1=6120 and r2=1394
0.0
0.0
0.0
0.0
0.0
549: RandomState r1=5912 and r2=4483
0.01
0.005
0.01
0.01
0.005
550: RandomState r1=582 and r2=337
0.0
0.0
0.0
0.0
0.0
551: RandomState r1=4635 and r2=1142
0.005
0.005
0.005
0.005
0.005
552: RandomState r1=4990 and r2=6798
0.0
0.0
0.0
0.0
0.0
553: RandomState r1=1080 and r2=2306
0.0
0.0
0.0
0.0
0.0
554: RandomState r1=3216 and r2=6295
0.005
0.005
0.005
0.005
0.005
555: RandomState r1=5308 and r2=138
0.005
0.0
0.0
0.0
0.0
556: RandomState r1=243

0.0
0.0
675: RandomState r1=3285 and r2=5363
0.0
0.0
0.0
0.0
0.0
676: RandomState r1=108 and r2=5884
0.005
0.0
0.0
0.0
0.0
677: RandomState r1=8468 and r2=8988
0.0
0.0
0.01
0.01
0.01
678: RandomState r1=4533 and r2=9979
0.01
0.005
0.005
0.005
0.005
679: RandomState r1=940 and r2=5369
0.005
0.005
0.01
0.005
0.005
680: RandomState r1=8875 and r2=3506
0.0
0.0
0.0
0.0
0.0
681: RandomState r1=3206 and r2=2025
0.01
0.005
0.01
0.01
0.005
682: RandomState r1=4637 and r2=4237
0.005
0.0
0.005
0.0
0.005
683: RandomState r1=4502 and r2=6421
0.0
0.0
0.0
0.0
0.0
684: RandomState r1=3491 and r2=2479
0.005
0.0
0.0
0.0
0.0
685: RandomState r1=7150 and r2=4960
0.01
0.01
0.01
0.01
0.005
686: RandomState r1=4466 and r2=4483
0.005
0.0
0.0
0.0
0.0
687: RandomState r1=8833 and r2=8591
0.0
0.0
0.005
0.005
0.0
688: RandomState r1=4090 and r2=7305
0.0
0.0
0.005
0.0
0.0
689: RandomState r1=1780 and r2=9167
0.005
0.005
0.005
0.005
0.005
690: RandomState r1=6803 and r2=2775
0.0
0.0
0.0
0.0
0.0
691: RandomState r1=

0.0
0.0
809: RandomState r1=5847 and r2=1812
0.0
0.0
0.005
0.005
0.005
810: RandomState r1=2445 and r2=4033
0.0
0.0
0.0
0.0
0.0
811: RandomState r1=4412 and r2=2233
0.0
0.0
0.005
0.005
0.0
812: RandomState r1=1978 and r2=3141
0.005
0.0
0.0
0.0
0.0
813: RandomState r1=1341 and r2=8809
0.005
0.005
0.005
0.005
0.0
814: RandomState r1=1020 and r2=9152
0.005
0.0
0.0
0.0
0.0
815: RandomState r1=1115 and r2=2562
0.015
0.01
0.01
0.01
0.01
816: RandomState r1=6962 and r2=8828
0.005
0.0
0.0
0.0
0.0
817: RandomState r1=9526 and r2=1352
0.005
0.005
0.005
0.005
0.005
818: RandomState r1=1790 and r2=7145
0.0
0.0
0.005
0.0
0.0
819: RandomState r1=2307 and r2=4327
0.0
0.0
0.005
0.005
0.0
820: RandomState r1=9859 and r2=3295
0.0
0.0
0.0
0.0
0.0
821: RandomState r1=9771 and r2=4650
0.005
0.005
0.005
0.005
0.005
822: RandomState r1=7669 and r2=1223
0.005
0.0
0.0
0.0
0.0
823: RandomState r1=4428 and r2=1542
0.0
0.0
0.0
0.0
0.0
824: RandomState r1=8609 and r2=3470
0.005
0.005
0.005
0.005
0.005
825: RandomS

0.0
0.0
944: RandomState r1=8732 and r2=3285
0.005
0.0
0.0
0.0
0.0
945: RandomState r1=620 and r2=3449
0.0
0.0
0.005
0.0
0.005
946: RandomState r1=8650 and r2=7111
0.0
0.0
0.0
0.0
0.0
947: RandomState r1=4955 and r2=1890
0.01
0.01
0.005
0.01
0.005
948: RandomState r1=3260 and r2=2016
0.005
0.005
0.005
0.005
0.005
949: RandomState r1=462 and r2=3162
0.005
0.005
0.005
0.01
0.005
950: RandomState r1=9737 and r2=3172
0.0
0.0
0.005
0.0
0.0
951: RandomState r1=138 and r2=563
0.0
0.0
0.005
0.005
0.0
952: RandomState r1=9669 and r2=9910
0.01
0.01
0.005
0.005
0.01
953: RandomState r1=5203 and r2=4993
0.0
0.0
0.0
0.0
0.005
954: RandomState r1=5387 and r2=7670
0.0
0.0
0.01
0.01
0.0
955: RandomState r1=6126 and r2=8963
0.005
0.0
0.005
0.005
0.0
956: RandomState r1=1741 and r2=2461
0.0
0.0
0.0
0.0
0.0
957: RandomState r1=5615 and r2=1725
0.0
0.0
0.0
0.0
0.0
958: RandomState r1=9215 and r2=2361
0.01
0.005
0.005
0.005
0.005
959: RandomState r1=9627 and r2=5684
0.005
0.005
0.005
0.005
0.005
960: Rando

In [14]:
print('Results without shadowing:')
print(1 - p)

Results without shadowing:
[[ 0.997055  0.99796   0.99664   0.996605  0.99753 ]
 [ 1.        1.        1.        1.        1.      ]
 [ 0.985     0.985     0.985     0.985     0.865   ]]


# Com shadowing

In [15]:
# Importing data

# importando os datasets
t2 = pd.read_csv('resultados/t2_OhBuildings_ComShadowing_Modificado', delimiter='\t')
t3 = pd.read_csv('resultados/t3_OhBuildings_ComShadowing_Modificado', delimiter='\t')

# garantindo que utilizaremos apenas as sementes presentes nos dois datasets
t2 = t2[t2.nRun.isin(t3.nRun)]
t3 = t3[t3.nRun.isin(t2.nRun)]
t2 = t2.reset_index(drop=True)
t3 = t3.reset_index(drop=True)

# combinando os datasets
data = t2
data = data.drop(['targetCellId', 'downloadTime', 'rxBytes'], axis=1)
data['downloadTimeT2'] = t2.downloadTime
data['downloadTimeT3'] = t3.downloadTime
data['rxBytesT2'] = t2.rxBytes
data['rxBytesT3'] = t3.rxBytes

data['best_output'] = data.apply(best_output, axis=1)
data.head()

# Splitting the data between independent variables and dependent variable
y = data['best_output']
x = data[['rsrp1','rsrq1','rsrp2','rsrq2','rsrp3','rsrq3','previousrsrp1','previousrsrq1','previousrsrp2','previousrsrq2','previousrsrp3','previousrsrq3']]

# Splitting test data and train data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)
# Feature scaling (normalizing or stadardization of the scales)
# Helps the conversion of the algorithm
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test) # There is no need to fit after the training set is fit

In [17]:
conf=gettingAllClassifiersConfigurations(x_train=x_train, y_train=y_train, randomState=0)

# Setting the classifiers
classifier_knn=conf[0]
classifier_svm=conf[1]
classifier_dt=conf[2]
classifier_rf=conf[3]
classifier_nn=conf[4]
#classifier_knn=KNeighborsClassifier(algorithm='auto',n_neighbors=5, p=1, weights='uniform')
#classifier_svm=SVC(C=200, decision_function_shape='ovo', degree=3, kernel='poly', shrinking=True)
#classifier_dt=DecisionTreeClassifier(class_weight= 'balanced', criterion= 'gini', max_features= None, splitter= 'best')
#classifier_rf=RandomForestClassifier(bootstrap= True, class_weight='balanced', criterion= 'gini', 
#                                     max_features= None, n_estimators= 2, n_jobs= -1, verbose= 0)


classifiers=[classifier_knn,classifier_svm,classifier_dt,classifier_rf, classifier_nn]

Best parameters for NN Classifier
MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=[10, 10, 10], learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
Best parameters for KNN Classifier
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=21, p=2,
           weights='uniform')
Best parameters for SVM Classifier
SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)
Best parameters for DT Classifier
DecisionTreeClassifier(class_wei

In [18]:
# Applying the classifiers
maxRange=1000
p = applyingAllClassifiers(numberOfRandomStates=maxRange, x=x, y=y, classifiers=classifiers)

0: RandomState r1=1783 and r2=3359
0.356164383562
0.180821917808
0.394520547945
0.405479452055
0.186301369863
1: RandomState r1=477 and r2=4050
0.383561643836
0.161643835616
0.397260273973
0.383561643836
0.197260273973
2: RandomState r1=5626 and r2=6489
0.358904109589
0.167123287671
0.38904109589
0.342465753425
0.156164383562
3: RandomState r1=5765 and r2=9232
0.331506849315
0.169863013699
0.378082191781
0.364383561644
0.169863013699
4: RandomState r1=2563 and r2=6275
0.361643835616
0.186301369863
0.342465753425
0.356164383562
0.2
5: RandomState r1=5127 and r2=2213
0.342465753425
0.197260273973
0.372602739726
0.331506849315
0.183561643836
6: RandomState r1=891 and r2=4064
0.339726027397
0.205479452055
0.378082191781
0.353424657534
0.139726027397
7: RandomState r1=2444 and r2=7169
0.339726027397
0.191780821918
0.364383561644
0.331506849315
0.158904109589
8: RandomState r1=6740 and r2=3862
0.350684931507
0.178082191781
0.331506849315
0.331506849315
0.150684931507
9: RandomState r1=8570 a

0.2
75: RandomState r1=9463 and r2=2419
0.353424657534
0.169863013699
0.358904109589
0.339726027397
0.194520547945
76: RandomState r1=5343 and r2=2314
0.358904109589
0.183561643836
0.378082191781
0.356164383562
0.169863013699
77: RandomState r1=7487 and r2=4360
0.331506849315
0.169863013699
0.394520547945
0.383561643836
0.186301369863
78: RandomState r1=3712 and r2=5562
0.339726027397
0.172602739726
0.367123287671
0.358904109589
0.178082191781
79: RandomState r1=4567 and r2=4757
0.380821917808
0.194520547945
0.419178082192
0.364383561644
0.191780821918
80: RandomState r1=3300 and r2=2060
0.369863013699
0.175342465753
0.375342465753
0.38904109589
0.158904109589
81: RandomState r1=2256 and r2=3396
0.353424657534
0.164383561644
0.375342465753
0.358904109589
0.18904109589
82: RandomState r1=5045 and r2=1460
0.369863013699
0.161643835616
0.38904109589
0.416438356164
0.178082191781
83: RandomState r1=8806 and r2=1618
0.317808219178
0.175342465753
0.32602739726
0.380821917808
0.153424657534
8

0.167123287671
149: RandomState r1=5967 and r2=290
0.312328767123
0.145205479452
0.339726027397
0.301369863014
0.167123287671
150: RandomState r1=8662 and r2=7681
0.323287671233
0.161643835616
0.369863013699
0.309589041096
0.158904109589
151: RandomState r1=7322 and r2=1854
0.334246575342
0.191780821918
0.367123287671
0.353424657534
0.180821917808
152: RandomState r1=8484 and r2=7043
0.334246575342
0.172602739726
0.367123287671
0.323287671233
0.142465753425
153: RandomState r1=396 and r2=4614
0.367123287671
0.161643835616
0.353424657534
0.350684931507
0.186301369863
154: RandomState r1=7454 and r2=6924
0.386301369863
0.167123287671
0.416438356164
0.408219178082
0.167123287671
155: RandomState r1=4810 and r2=9789
0.328767123288
0.175342465753
0.372602739726
0.315068493151
0.172602739726
156: RandomState r1=2228 and r2=8940
0.342465753425
0.2
0.375342465753
0.298630136986
0.147945205479
157: RandomState r1=7906 and r2=9566
0.356164383562
0.194520547945
0.430136986301
0.375342465753
0.213

0.194520547945
0.421917808219
0.402739726027
0.221917808219
224: RandomState r1=1992 and r2=6562
0.361643835616
0.180821917808
0.347945205479
0.317808219178
0.186301369863
225: RandomState r1=1757 and r2=7175
0.345205479452
0.197260273973
0.347945205479
0.309589041096
0.197260273973
226: RandomState r1=1135 and r2=1956
0.358904109589
0.186301369863
0.361643835616
0.375342465753
0.18904109589
227: RandomState r1=4792 and r2=1597
0.347945205479
0.167123287671
0.331506849315
0.350684931507
0.158904109589
228: RandomState r1=2486 and r2=2867
0.298630136986
0.175342465753
0.364383561644
0.347945205479
0.172602739726
229: RandomState r1=2319 and r2=3007
0.386301369863
0.216438356164
0.397260273973
0.345205479452
0.213698630137
230: RandomState r1=4520 and r2=3066
0.339726027397
0.169863013699
0.4
0.353424657534
0.169863013699
231: RandomState r1=7966 and r2=5182
0.364383561644
0.194520547945
0.375342465753
0.383561643836
0.183561643836
232: RandomState r1=2728 and r2=3086
0.33698630137
0.180

0.353424657534
0.191780821918
298: RandomState r1=2823 and r2=6373
0.361643835616
0.186301369863
0.353424657534
0.350684931507
0.164383561644
299: RandomState r1=3873 and r2=7675
0.353424657534
0.219178082192
0.372602739726
0.342465753425
0.183561643836
300: RandomState r1=4480 and r2=1129
0.361643835616
0.194520547945
0.416438356164
0.331506849315
0.172602739726
301: RandomState r1=2045 and r2=5779
0.378082191781
0.2
0.378082191781
0.369863013699
0.180821917808
302: RandomState r1=5249 and r2=4239
0.375342465753
0.169863013699
0.380821917808
0.361643835616
0.178082191781
303: RandomState r1=6199 and r2=9472
0.394520547945
0.180821917808
0.375342465753
0.364383561644
0.224657534247
304: RandomState r1=8965 and r2=5475
0.32602739726
0.164383561644
0.309589041096
0.378082191781
0.175342465753
305: RandomState r1=3566 and r2=4755
0.380821917808
0.172602739726
0.367123287671
0.33698630137
0.167123287671
306: RandomState r1=5165 and r2=1402
0.347945205479
0.158904109589
0.378082191781
0.336

...

0.361643835616
0.172602739726
670: RandomState r1=6391 and r2=9029
0.347945205479
0.172602739726
0.394520547945
0.342465753425
0.161643835616
671: RandomState r1=8593 and r2=7897
0.342465753425
0.172602739726
0.41095890411
0.339726027397
0.172602739726
672: RandomState r1=3544 and r2=7621
0.383561643836
0.191780821918
0.353424657534
0.33698630137
0.18904109589
673: RandomState r1=7548 and r2=4735
0.350684931507
0.172602739726
0.364383561644
0.361643835616
0.150684931507
674: RandomState r1=7971 and r2=4486
0.328767123288
0.178082191781
0.367123287671
0.361643835616
0.150684931507
675: RandomState r1=9696 and r2=7379
0.364383561644
0.158904109589
0.38904109589
0.33698630137
0.167123287671
676: RandomState r1=5939 and r2=6422
0.372602739726
0.183561643836
0.364383561644
0.342465753425
0.208219178082
677: RandomState r1=7208 and r2=1935
0.33698630137
0.186301369863
0.372602739726
0.345205479452
0.175342465753
678: RandomState r1=2309 and r2=1604
0.315068493151
0.169863013699
0.38356164383

0.372602739726
0.178082191781
744: RandomState r1=2724 and r2=5581
0.347945205479
0.18904109589
0.38904109589
0.32602739726
0.183561643836
745: RandomState r1=8931 and r2=3992
0.364383561644
0.167123287671
0.4
0.372602739726
0.186301369863
746: RandomState r1=6097 and r2=1753
0.353424657534
0.180821917808
0.38904109589
0.328767123288
0.194520547945
747: RandomState r1=2764 and r2=1532
0.350684931507
0.175342465753
0.391780821918
0.345205479452
0.147945205479
748: RandomState r1=3389 and r2=1474
0.328767123288
0.18904109589
0.339726027397
0.356164383562
0.145205479452
749: RandomState r1=3691 and r2=8756
0.367123287671
0.191780821918
0.372602739726
0.356164383562
0.172602739726
750: RandomState r1=7084 and r2=8115
0.32602739726
0.180821917808
0.342465753425
0.339726027397
0.175342465753
751: RandomState r1=5216 and r2=5344
0.315068493151
0.194520547945
0.402739726027
0.353424657534
0.202739726027
752: RandomState r1=5985 and r2=4283
0.32602739726
0.197260273973
0.369863013699
0.37808219

0.18904109589
818: RandomState r1=1056 and r2=4864
0.342465753425
0.180821917808
0.383561643836
0.350684931507
0.202739726027
819: RandomState r1=6006 and r2=9187
0.350684931507
0.2
0.386301369863
0.347945205479
0.139726027397
820: RandomState r1=3018 and r2=7544
0.358904109589
0.178082191781
0.356164383562
0.369863013699
0.158904109589
821: RandomState r1=3443 and r2=3900
0.334246575342
0.169863013699
0.358904109589
0.328767123288
0.191780821918
822: RandomState r1=5455 and r2=8606
0.33698630137
0.183561643836
0.328767123288
0.339726027397
0.180821917808
823: RandomState r1=3700 and r2=4108
0.347945205479
0.186301369863
0.33698630137
0.295890410959
0.147945205479
824: RandomState r1=5032 and r2=552
0.328767123288
0.158904109589
0.386301369863
0.347945205479
0.194520547945
825: RandomState r1=8268 and r2=347
0.383561643836
0.21095890411
0.380821917808
0.364383561644
0.230136986301
826: RandomState r1=372 and r2=896
0.372602739726
0.156164383562
0.408219178082
0.33698630137
0.1890410958

0.169863013699
0.347945205479
0.331506849315
0.153424657534
893: RandomState r1=9545 and r2=4104
0.347945205479
0.147945205479
0.361643835616
0.367123287671
0.169863013699
894: RandomState r1=1417 and r2=5280
0.361643835616
0.175342465753
0.369863013699
0.361643835616
0.178082191781
895: RandomState r1=4275 and r2=8122
0.339726027397
0.194520547945
0.364383561644
0.353424657534
0.180821917808
896: RandomState r1=1134 and r2=7161
0.38904109589
0.205479452055
0.408219178082
0.306849315068
0.150684931507
897: RandomState r1=4145 and r2=8228
0.375342465753
0.164383561644
0.375342465753
0.391780821918
0.186301369863
898: RandomState r1=4354 and r2=4597
0.356164383562
0.172602739726
0.353424657534
0.375342465753
0.186301369863
899: RandomState r1=4991 and r2=7538
0.367123287671
0.205479452055
0.402739726027
0.317808219178
0.172602739726
900: RandomState r1=9291 and r2=1608
0.358904109589
0.197260273973
0.416438356164
0.353424657534
0.21095890411
901: RandomState r1=9163 and r2=9238
0.3698630

0.221917808219
0.397260273973
0.364383561644
0.18904109589
967: RandomState r1=6104 and r2=1302
0.356164383562
0.194520547945
0.367123287671
0.361643835616
0.2
968: RandomState r1=3095 and r2=4624
0.367123287671
0.205479452055
0.391780821918
0.350684931507
0.161643835616
969: RandomState r1=8409 and r2=9840
0.369863013699
0.213698630137
0.375342465753
0.369863013699
0.21095890411
970: RandomState r1=2439 and r2=4071
0.347945205479
0.167123287671
0.41095890411
0.356164383562
0.142465753425
971: RandomState r1=6975 and r2=7297
0.328767123288
0.153424657534
0.369863013699
0.350684931507
0.12602739726
972: RandomState r1=2502 and r2=4551
0.380821917808
0.18904109589
0.375342465753
0.358904109589
0.21095890411
973: RandomState r1=8593 and r2=4328
0.342465753425
0.172602739726
0.41095890411
0.339726027397
0.172602739726
974: RandomState r1=3656 and r2=4123
0.353424657534
0.134246575342
0.361643835616
0.334246575342
0.147945205479
975: RandomState r1=5329 and r2=3043
0.339726027397
0.15890410

In [19]:
print('Results with shadowing:')
print(1 - p)

Results with shadowing:
[[ 0.64813425  0.81608493  0.62840822  0.65247945  0.82368767]
 [ 0.73150685  0.86849315  0.70958904  0.73424658  0.87945205]
 [ 0.56986301  0.75890411  0.55890411  0.57534247  0.75616438]]
